<a href="https://colab.research.google.com/github/azzbc7819/test_igaimers/blob/develop/XGBoost_threshold_%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# Plotly 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
trainfile = '/content/drive/MyDrive/LG Aimers 해커톤/train.csv'
df = pd.read_csv(trainfile)
df.head()

#데이터 로드

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [4]:
columns_to_drop = [ 'PRODUCT_ID','TIMESTAMP', 'X_3', 'X_4', 'X_6', 'X_10', 'X_14', 'X_19', 'X_23', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_67', 'X_68', 'X_69', 'X_70', 'X_71', 'X_72', 'X_74', 'X_75', 'X_76', 'X_77', 'X_78', 'X_79', 'X_80', 'X_81', 'X_82', 'X_83', 'X_84', 'X_85', 'X_89', 'X_91', 'X_92', 'X_93', 'X_96', 'X_100', 'X_104', 'X_108', 'X_112', 'X_116', 'X_122', 'X_130', 'X_131', 'X_135', 'X_142', 'X_145', 'X_147', 'X_154', 'X_158', 'X_162', 'X_166', 'X_170', 'X_174', 'X_178', 'X_182', 'X_186', 'X_187', 'X_188', 'X_190', 'X_191', 'X_192', 'X_193', 'X_194', 'X_195', 'X_232', 'X_234', 'X_235', 'X_239', 'X_241', 'X_244', 'X_249', 'X_252', 'X_255', 'X_259', 'X_260', 'X_264', 'X_274', 'X_278', 'X_282', 'X_312', 'X_313', 'X_314', 'X_315', 'X_316', 'X_317', 'X_319', 'X_322', 'X_323', 'X_324', 'X_325', 'X_326', 'X_327', 'X_328', 'X_329', 'X_330', 'X_334', 'X_336', 'X_337', 'X_338', 'X_339', 'X_340', 'X_342', 'X_343', 'X_347', 'X_351', 'X_355', 'X_359', 'X_360', 'X_361', 'X_362', 'X_363', 'X_369', 'X_371', 'X_372', 'X_375', 'X_376', 'X_377', 'X_378', 'X_381', 'X_382', 'X_383', 'X_390', 'X_391', 'X_392', 'X_409', 'X_410', 'X_411', 'X_424', 'X_425', 'X_426', 'X_427', 'X_429', 'X_430', 'X_431', 'X_432', 'X_433', 'X_434', 'X_466', 'X_467', 'X_480', 'X_487', 'X_503', 'X_505', 'X_508', 'X_512', 'X_519', 'X_551', 'X_558', 'X_561', 'X_567', 'X_583', 'X_617', 'X_620', 'X_631', 'X_633', 'X_634', 'X_636', 'X_638', 'X_639', 'X_640', 'X_641', 'X_642', 'X_650', 'X_672', 'X_673', 'X_676', 'X_691', 'X_692', 'X_695', 'X_699', 'X_715', 'X_729', 'X_732', 'X_743', 'X_749', 'X_759', 'X_760', 'X_761', 'X_764', 'X_776', 'X_777', 'X_778', 'X_836', 'X_843', 'X_844', 'X_849', 'X_859', 'X_872', 'X_873', 'X_874', 'X_879', 'X_880', 'X_881', 'X_886', 'X_887', 'X_888', 'X_889', 'X_910', 'X_912', 'X_934', 'X_935', 'X_936', 'X_937', 'X_950', 'X_951', 'X_952', 'X_969', 'X_970', 'X_971', 'X_987', 'X_988', 'X_989', 'X_992', 'X_1003', 'X_1004', 'X_1005', 'X_1006', 'X_1007', 'X_1008', 'X_1020', 'X_1021', 'X_1022', 'X_1023', 'X_1024', 'X_1025', 'X_1070', 'X_1092', 'X_1103', 'X_1119', 'X_1130', 'X_1137', 'X_1146', 'X_1157', 'X_1206', 'X_1216', 'X_1219', 'X_1248', 'X_1249', 'X_1250', 'X_1251', 'X_1252', 'X_1253', 'X_1255', 'X_1293', 'X_1298', 'X_1309', 'X_1311', 'X_1312', 'X_1314', 'X_1316', 'X_1317', 'X_1318', 'X_1319', 'X_1320', 'X_1328', 'X_1361', 'X_1362', 'X_1363', 'X_1364', 'X_1367', 'X_1392', 'X_1393', 'X_1394', 'X_1395', 'X_1396', 'X_1399', 'X_1407', 'X_1426', 'X_1457', 'X_1487', 'X_1502', 'X_1503', 'X_1504', 'X_1522', 'X_1523', 'X_1524', 'X_1525', 'X_1531', 'X_1537', 'X_1540', 'X_1571', 'X_1572', 'X_1573', 'X_1574', 'X_1575', 'X_1576', 'X_1577', 'X_1578', 'X_1579', 'X_1580', 'X_1581', 'X_1582', 'X_1644', 'X_1645', 'X_1675', 'X_1676', 'X_1677', 'X_1680', 'X_1681', 'X_1683', 'X_1684', 'X_1685', 'X_1687', 'X_1688', 'X_1689', 'X_1691', 'X_1692', 'X_1693', 'X_1695', 'X_1698', 'X_1699', 'X_1700', 'X_1701', 'X_1702', 'X_1707', 'X_1708', 'X_1709', 'X_1710', 'X_1711', 'X_1719', 'X_1721', 'X_1722', 'X_1723', 'X_1724', 'X_1729', 'X_1731', 'X_1733', 'X_1749', 'X_1753', 'X_1754', 'X_1759', 'X_1760', 'X_1765', 'X_1766', 'X_1771', 'X_1772', 'X_1777', 'X_1778', 'X_1783', 'X_1784', 'X_1789', 'X_1790', 'X_1795', 'X_1796', 'X_1801', 'X_1802', 'X_1803', 'X_1807', 'X_1808', 'X_1811', 'X_1823', 'X_1827', 'X_1828', 'X_1835', 'X_1836', 'X_1837', 'X_1838', 'X_1839', 'X_1840', 'X_1841', 'X_1842', 'X_1844', 'X_1845', 'X_1846', 'X_1847', 'X_1848', 'X_1851', 'X_1852', 'X_1869', 'X_1870', 'X_1871', 'X_1872', 'X_1969', 'X_1970', 'X_1971', 'X_1972', 'X_1973', 'X_1974', 'X_1975', 'X_1976', 'X_1977', 'X_1978', 'X_1979', 'X_1980', 'X_1981', 'X_1982', 'X_1983', 'X_1984', 'X_1985', 'X_1986', 'X_1987', 'X_1988', 'X_1989', 'X_1990', 'X_1991', 'X_1992', 'X_1993', 'X_1994', 'X_1995', 'X_1996', 'X_1997', 'X_1998', 'X_1999', 'X_2000', 'X_2001', 'X_2002', 'X_2003', 'X_2004', 'X_2005', 'X_2006', 'X_2007', 'X_2008', 'X_2009', 'X_2010', 'X_2011', 'X_2012', 'X_2013', 'X_2014', 'X_2015', 'X_2016', 'X_2029', 'X_2030', 'X_2031', 'X_2032', 'X_2033', 'X_2034', 'X_2035', 'X_2036', 'X_2037', 'X_2038', 'X_2039', 'X_2040', 'X_2052', 'X_2053', 'X_2054', 'X_2055', 'X_2069', 'X_2077', 'X_2085', 'X_2092', 'X_2188', 'X_2189', 'X_2417', 'X_2419', 'X_2420', 'X_2425', 'X_2462', 'X_2540', 'X_2541', 'X_2542', 'X_2545', 'X_2548', 'X_2549', 'X_2550', 'X_2551', 'X_2628', 'X_2629', 'X_2630', 'X_2631', 'X_2632', 'X_2633', 'X_2634', 'X_2635', 'X_2636', 'X_2637', 'X_2638', 'X_2639', 'X_2640', 'X_2641', 'X_2642', 'X_2643', 'X_2644', 'X_2645', 'X_2646', 'X_2647', 'X_2648', 'X_2649', 'X_2650', 'X_2651', 'X_2652', 'X_2653', 'X_2654', 'X_2655', 'X_2656', 'X_2657', 'X_2658', 'X_2659', 'X_2660', 'X_2661', 'X_2662', 'X_2663', 'X_2664', 'X_2665', 'X_2666', 'X_2667', 'X_2668', 'X_2669', 'X_2670', 'X_2671', 'X_2672', 'X_2673', 'X_2674', 'X_2675', 'X_2676', 'X_2677', 'X_2678', 'X_2679', 'X_2680', 'X_2681', 'X_2682', 'X_2683', 'X_2684', 'X_2685', 'X_2686', 'X_2687', 'X_2688', 'X_2689', 'X_2690', 'X_2691', 'X_2692', 'X_2693', 'X_2694', 'X_2695', 'X_2696', 'X_2697', 'X_2698', 'X_2699', 'X_2721', 'X_2722', 'X_2723', 'X_2730', 'X_2732', 'X_2733', 'X_2738', 'X_2773', 'X_2774', 'X_2775', 'X_2838', 'X_2844', 'X_2871', 'X_2872', 'X_2873', 'X_2874', 'X_2875']

x_data = df.drop(columns = columns_to_drop)

#nuniq()썼을 때 1이하인 컬럼 및 학습에 의미 없는 컬럼 드랍

In [5]:
df_test = pd.read_csv('/content/drive/MyDrive/LG Aimers 해커톤/test.csv')
df_test.set_index('PRODUCT_ID', inplace = True)

columns_to_drop = ['TIMESTAMP', 'X_3', 'X_4', 'X_6', 'X_10', 'X_14', 'X_19', 'X_23', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_67', 'X_68', 'X_69', 'X_70', 'X_71', 'X_72', 'X_74', 'X_75', 'X_76', 'X_77', 'X_78', 'X_79', 'X_80', 'X_81', 'X_82', 'X_83', 'X_84', 'X_85', 'X_89', 'X_91', 'X_92', 'X_93', 'X_96', 'X_100', 'X_104', 'X_108', 'X_112', 'X_116', 'X_122', 'X_130', 'X_131', 'X_135', 'X_142', 'X_145', 'X_147', 'X_154', 'X_158', 'X_162', 'X_166', 'X_170', 'X_174', 'X_178', 'X_182', 'X_186', 'X_187', 'X_188', 'X_190', 'X_191', 'X_192', 'X_193', 'X_194', 'X_195', 'X_232', 'X_234', 'X_235', 'X_239', 'X_241', 'X_244', 'X_249', 'X_252', 'X_255', 'X_259', 'X_260', 'X_264', 'X_274', 'X_278', 'X_282', 'X_312', 'X_313', 'X_314', 'X_315', 'X_316', 'X_317', 'X_319', 'X_322', 'X_323', 'X_324', 'X_325', 'X_326', 'X_327', 'X_328', 'X_329', 'X_330', 'X_334', 'X_336', 'X_337', 'X_338', 'X_339', 'X_340', 'X_342', 'X_343', 'X_347', 'X_351', 'X_355', 'X_359', 'X_360', 'X_361', 'X_362', 'X_363', 'X_369', 'X_371', 'X_372', 'X_375', 'X_376', 'X_377', 'X_378', 'X_381', 'X_382', 'X_383', 'X_390', 'X_391', 'X_392', 'X_409', 'X_410', 'X_411', 'X_424', 'X_425', 'X_426', 'X_427', 'X_429', 'X_430', 'X_431', 'X_432', 'X_433', 'X_434', 'X_466', 'X_467', 'X_480', 'X_487', 'X_503', 'X_505', 'X_508', 'X_512', 'X_519', 'X_551', 'X_558', 'X_561', 'X_567', 'X_583', 'X_617', 'X_620', 'X_631', 'X_633', 'X_634', 'X_636', 'X_638', 'X_639', 'X_640', 'X_641', 'X_642', 'X_650', 'X_672', 'X_673', 'X_676', 'X_691', 'X_692', 'X_695', 'X_699', 'X_715', 'X_729', 'X_732', 'X_743', 'X_749', 'X_759', 'X_760', 'X_761', 'X_764', 'X_776', 'X_777', 'X_778', 'X_836', 'X_843', 'X_844', 'X_849', 'X_859', 'X_872', 'X_873', 'X_874', 'X_879', 'X_880', 'X_881', 'X_886', 'X_887', 'X_888', 'X_889', 'X_910', 'X_912', 'X_934', 'X_935', 'X_936', 'X_937', 'X_950', 'X_951', 'X_952', 'X_969', 'X_970', 'X_971', 'X_987', 'X_988', 'X_989', 'X_992', 'X_1003', 'X_1004', 'X_1005', 'X_1006', 'X_1007', 'X_1008', 'X_1020', 'X_1021', 'X_1022', 'X_1023', 'X_1024', 'X_1025', 'X_1070', 'X_1092', 'X_1103', 'X_1119', 'X_1130', 'X_1137', 'X_1146', 'X_1157', 'X_1206', 'X_1216', 'X_1219', 'X_1248', 'X_1249', 'X_1250', 'X_1251', 'X_1252', 'X_1253', 'X_1255', 'X_1293', 'X_1298', 'X_1309', 'X_1311', 'X_1312', 'X_1314', 'X_1316', 'X_1317', 'X_1318', 'X_1319', 'X_1320', 'X_1328', 'X_1361', 'X_1362', 'X_1363', 'X_1364', 'X_1367', 'X_1392', 'X_1393', 'X_1394', 'X_1395', 'X_1396', 'X_1399', 'X_1407', 'X_1426', 'X_1457', 'X_1487', 'X_1502', 'X_1503', 'X_1504', 'X_1522', 'X_1523', 'X_1524', 'X_1525', 'X_1531', 'X_1537', 'X_1540', 'X_1571', 'X_1572', 'X_1573', 'X_1574', 'X_1575', 'X_1576', 'X_1577', 'X_1578', 'X_1579', 'X_1580', 'X_1581', 'X_1582', 'X_1644', 'X_1645', 'X_1675', 'X_1676', 'X_1677', 'X_1680', 'X_1681', 'X_1683', 'X_1684', 'X_1685', 'X_1687', 'X_1688', 'X_1689', 'X_1691', 'X_1692', 'X_1693', 'X_1695', 'X_1698', 'X_1699', 'X_1700', 'X_1701', 'X_1702', 'X_1707', 'X_1708', 'X_1709', 'X_1710', 'X_1711', 'X_1719', 'X_1721', 'X_1722', 'X_1723', 'X_1724', 'X_1729', 'X_1731', 'X_1733', 'X_1749', 'X_1753', 'X_1754', 'X_1759', 'X_1760', 'X_1765', 'X_1766', 'X_1771', 'X_1772', 'X_1777', 'X_1778', 'X_1783', 'X_1784', 'X_1789', 'X_1790', 'X_1795', 'X_1796', 'X_1801', 'X_1802', 'X_1803', 'X_1807', 'X_1808', 'X_1811', 'X_1823', 'X_1827', 'X_1828', 'X_1835', 'X_1836', 'X_1837', 'X_1838', 'X_1839', 'X_1840', 'X_1841', 'X_1842', 'X_1844', 'X_1845', 'X_1846', 'X_1847', 'X_1848', 'X_1851', 'X_1852', 'X_1869', 'X_1870', 'X_1871', 'X_1872', 'X_1969', 'X_1970', 'X_1971', 'X_1972', 'X_1973', 'X_1974', 'X_1975', 'X_1976', 'X_1977', 'X_1978', 'X_1979', 'X_1980', 'X_1981', 'X_1982', 'X_1983', 'X_1984', 'X_1985', 'X_1986', 'X_1987', 'X_1988', 'X_1989', 'X_1990', 'X_1991', 'X_1992', 'X_1993', 'X_1994', 'X_1995', 'X_1996', 'X_1997', 'X_1998', 'X_1999', 'X_2000', 'X_2001', 'X_2002', 'X_2003', 'X_2004', 'X_2005', 'X_2006', 'X_2007', 'X_2008', 'X_2009', 'X_2010', 'X_2011', 'X_2012', 'X_2013', 'X_2014', 'X_2015', 'X_2016', 'X_2029', 'X_2030', 'X_2031', 'X_2032', 'X_2033', 'X_2034', 'X_2035', 'X_2036', 'X_2037', 'X_2038', 'X_2039', 'X_2040', 'X_2052', 'X_2053', 'X_2054', 'X_2055', 'X_2069', 'X_2077', 'X_2085', 'X_2092', 'X_2188', 'X_2189', 'X_2417', 'X_2419', 'X_2420', 'X_2425', 'X_2462', 'X_2540', 'X_2541', 'X_2542', 'X_2545', 'X_2548', 'X_2549', 'X_2550', 'X_2551', 'X_2628', 'X_2629', 'X_2630', 'X_2631', 'X_2632', 'X_2633', 'X_2634', 'X_2635', 'X_2636', 'X_2637', 'X_2638', 'X_2639', 'X_2640', 'X_2641', 'X_2642', 'X_2643', 'X_2644', 'X_2645', 'X_2646', 'X_2647', 'X_2648', 'X_2649', 'X_2650', 'X_2651', 'X_2652', 'X_2653', 'X_2654', 'X_2655', 'X_2656', 'X_2657', 'X_2658', 'X_2659', 'X_2660', 'X_2661', 'X_2662', 'X_2663', 'X_2664', 'X_2665', 'X_2666', 'X_2667', 'X_2668', 'X_2669', 'X_2670', 'X_2671', 'X_2672', 'X_2673', 'X_2674', 'X_2675', 'X_2676', 'X_2677', 'X_2678', 'X_2679', 'X_2680', 'X_2681', 'X_2682', 'X_2683', 'X_2684', 'X_2685', 'X_2686', 'X_2687', 'X_2688', 'X_2689', 'X_2690', 'X_2691', 'X_2692', 'X_2693', 'X_2694', 'X_2695', 'X_2696', 'X_2697', 'X_2698', 'X_2699', 'X_2721', 'X_2722', 'X_2723', 'X_2730', 'X_2732', 'X_2733', 'X_2738', 'X_2773', 'X_2774', 'X_2775', 'X_2838', 'X_2844', 'X_2871', 'X_2872', 'X_2873', 'X_2874', 'X_2875']
df_test = df_test.drop(columns = columns_to_drop)

In [6]:
# 범주형 데이터 > 수치형 데이터로 전환

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(x_data[i])
    x_data[i] = le.transform(x_data[i])

    for label in np.unique(df_test[i]): 
      if label not in le.classes_: 
        le.classes_ = np.append(le.classes_, label)
    df_test[i] = le.transform(df_test[i]) 


In [9]:
for columns_name in x_data:
  x_data.fillna({columns_name : 0 }, inplace=True)

#0으로 채우기, dataframe 형태 유지 하기 위해 imputer 쓰지 않음

In [14]:
y_feat_corr = pd.DataFrame(x_data.corr()['Y_Class'])

In [25]:
y_feat_corr = y_feat_corr.T.drop(columns = ['Y_Class', 'Y_Quality']).T

KeyError: ignored

In [31]:
y_feat_corr.describe() #이렇게 볼 때 Y_Class와 상관계수가 높아봤자 0.26

,Y_Class
count,2325.000000
mean,0.010868
std,0.076421
min,-0.193561
25%,-0.050823
50%,-0.015983
75%,0.085003
max,0.264064


In [37]:
y_feat_corr_mean = y_feat_corr.describe().loc['mean'][0]

In [38]:
print(y_feat_corr_mean)

0.010867627795239677


In [79]:
def remove_collinear_features(x, y_feat_corr,threshold):
    # 데이터프레임 x 의 상관계수 구하기
    corr_matrix = y_feat_corr
    iters = range(len(corr_matrix))
    # 제거할 컬럼들 저장할 리스트
    drop_cols = []

    for i in iters:

      # j행 (i+1)열 상관계수 가져오기
      item = corr_matrix.iloc[i]
      # 상관계수 셀의 컬럼명 가져오기
      col = 'Y_Class'
      # 상관계수 셀의 행 인덱스 가져오기
      row = item.name
      # 상관계수
      val = item[0]

      if val <= threshold:
        print(col, "열", row, "행의 상관계수 : ", val)
        drop_cols.append(row)

    drops = set(drop_cols)
    x = x.drop(columns=drops)

    return x

In [50]:
y_feat_corr

,Y_Class
LINE,-0.016027
PRODUCT_CODE,-0.034240
X_1,-0.003244
X_2,-0.031998
X_5,-0.032277
...,...
X_2866,-0.111978
X_2867,-0.109532
X_2868,-0.096989
X_2869,-0.122089


In [80]:
threshold = y_feat_corr_mean #threshold를 y랑 feature들 사이 평균으로 둠으로써 이보다 작은 상관계수를 가진 칼람 삭제
data = remove_collinear_features(x_data,y_feat_corr, threshold)

Y_Class 열 LINE 행의 상관계수 :  -0.016027096114645853
Y_Class 열 PRODUCT_CODE 행의 상관계수 :  -0.034240159050846035
Y_Class 열 X_1 행의 상관계수 :  -0.003244478518455633
Y_Class 열 X_2 행의 상관계수 :  -0.03199815200340646
Y_Class 열 X_5 행의 상관계수 :  -0.03227740735904341
Y_Class 열 X_7 행의 상관계수 :  -0.024050948322521526
Y_Class 열 X_8 행의 상관계수 :  -0.026872368784176072
Y_Class 열 X_9 행의 상관계수 :  -0.012412937873735524
Y_Class 열 X_11 행의 상관계수 :  -0.02708206965767485
Y_Class 열 X_12 행의 상관계수 :  -0.027058740652506418
Y_Class 열 X_13 행의 상관계수 :  -0.027425900559066064
Y_Class 열 X_15 행의 상관계수 :  -0.0018182977521344543
Y_Class 열 X_16 행의 상관계수 :  -0.028556370030847995
Y_Class 열 X_17 행의 상관계수 :  -0.02856617389108163
Y_Class 열 X_18 행의 상관계수 :  -0.028572844758019304
Y_Class 열 X_20 행의 상관계수 :  -0.029965888904792154
Y_Class 열 X_21 행의 상관계수 :  -0.029553527855729365
Y_Class 열 X_22 행의 상관계수 :  -0.030068648438245
Y_Class 열 X_24 행의 상관계수 :  -0.0271531914136258
Y_Class 열 X_38 행의 상관계수 :  -0.030446153871139282
Y_Class 열 X_39 행의 상관계수 :  -0.03424015905084603

In [81]:
data

,Y_Class,Y_Quality,X_94,X_95,X_126,X_127,X_246,X_247,X_248,X_250,...,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865
0,1,0.533433,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,181.6,139.6,131.646667,115.4,209.0,197.286667,189.0,383.0,368.296296,353.0
1,2,0.541819,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,177.1,145.5,128.748276,119.7,198.0,193.296552,185.6,383.0,367.735849,353.0
2,1,0.531267,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,183.6,128.0,115.365517,104.0,193.4,179.820690,165.5,383.0,367.320755,353.0
3,2,0.537325,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,179.5,126.2,112.082759,94.5,190.3,181.920690,165.8,384.0,369.188679,353.0
4,1,0.531590,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,188.0,152.1,138.066667,109.7,208.6,196.393333,182.6,383.0,367.351852,352.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,1,0.526546,0.19,0.19,6608.0,6608.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
594,0,0.524022,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,168.2,185.1,159.527586,144.2,198.3,180.810345,168.7,384.0,369.811321,353.0
595,0,0.521289,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,153.4,187.6,178.248276,159.6,186.5,176.486207,156.6,383.0,367.018868,352.0
596,1,0.531375,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [82]:
y = data.pop('Y_Class')
x = data.drop(columns = ['Y_Quality'])

In [83]:
x

,X_94,X_95,X_126,X_127,X_246,X_247,X_248,X_250,X_251,X_253,...,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865
0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,181.6,139.6,131.646667,115.4,209.0,197.286667,189.0,383.0,368.296296,353.0
1,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,177.1,145.5,128.748276,119.7,198.0,193.296552,185.6,383.0,367.735849,353.0
2,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,183.6,128.0,115.365517,104.0,193.4,179.820690,165.5,383.0,367.320755,353.0
3,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,179.5,126.2,112.082759,94.5,190.3,181.920690,165.8,384.0,369.188679,353.0
4,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,188.0,152.1,138.066667,109.7,208.6,196.393333,182.6,383.0,367.351852,352.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.19,0.19,6608.0,6608.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
594,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,168.2,185.1,159.527586,144.2,198.3,180.810345,168.7,384.0,369.811321,353.0
595,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,153.4,187.6,178.248276,159.6,186.5,176.486207,156.6,383.0,367.018868,352.0
596,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [84]:
y

0      1
1      2
2      1
3      2
4      1
      ..
593    1
594    0
595    0
596    1
597    1
Name: Y_Class, Length: 598, dtype: int64

In [89]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings(action='ignore')

def get_score(y_data, x_data, model = 'XGB', imputer = 'simple', strategy = 'constant', fill_value = 0, n_neighbors = 3):

    if imputer == "simple":
        i = SimpleImputer(strategy=strategy, fill_value=fill_value)
    elif imputer == "knn":
        i = KNNImputer(n_neighbors=n_neighbors)
    else:
        return "INVALID ERROR"

    x_data = i.fit_transform(x_data)

    #데이터 클래스 불균형을 해결하기 위한 resampling 모델설정
    sm = SMOTE()

    # 데이터를 넣어 복제함
    X_resampled, y_resampled = sm.fit_resample(x_data,y_data)

    x_train, x_test, y_train, y_test = train_test_split(
        X_resampled,
        y_resampled,
        test_size = 0.2,
        random_state = 0,
        stratify = y_resampled
    )

    # sc = StandardScaler()
    # x_train = sc.fit_transform(x_train)
    # x_test = sc.transform(x_test)

    if model == "XGB":
        clf = XGBClassifier(random_state = 0, n_estimators=200, early_stopping_rounds=100, learning_rate=0.2)
    else:
        return "INVALID ERROR"

    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    return f1_score(y_pred, y_test, average="macro")

In [90]:
get_score(y, x) #상관계수 평균 이하인 컬럼 드랍 후 학습 결과, 컬럼 드랍하기 이전보다 성능 잘 안 나옴

0.8776244494788307